## RoBERTa Prediction code snippet

In [96]:
import sys
sys.path.append('/data/szr207/projects/SeerBERT/RoBERTa/fairseq')

from fairseq.models.roberta import RobertaModel
from tqdm.notebook import tqdm
import jsonlines
import os

In [98]:
roberta = RobertaModel.from_pretrained(
    '/data/szr207/projects/SeerBERT/RoBERTa/fairseq/checkpoints/',
    checkpoint_file='checkpoint1.pt',
    data_name_or_path='/data/szr207/projects/SeerBERT/RoBERTa/fairseq/QQP-bin'
)

roberta.cuda()

roberta.eval()

RobertaHubInterface(
  (model): RobertaModel(
    (decoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (embed_tokens): Embedding(50265, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 768, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (self_attn): MultiheadAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
            (final_layer_norm): LayerNorm((768

In [6]:
with open('/data/szr207/projects/SeerBERT/RoBERTa/fairseq/glue_data/QQP/test.tsv') as fin:
    for index, line in tqdm(enumerate(fin)):
        try:
            tokens = line.strip().split('\t')
            qid, aid, sent1, sent2, target = tokens[1], tokens[2], tokens[3], tokens[4], tokens[5]
            tokens = roberta.encode(sent1, sent2)
            prediction = roberta.predict('sentence_classification_head', tokens)
#             print(prediction)
            if prediction==1:
                print(qid, aid)
#             prediction_label = label_fn(prediction)
            ncorrect += int(prediction == target)
            nsamples += 1
        except:
            continue

550it [00:08, 65.79it/s]


KeyboardInterrupt: 

In [99]:
def reduce_str(body, N):
    if len(body.split()) > N:
        return ' '.join(body.split()[:N])
    return body

# reduce_str(body,254)

In [100]:
path_run = '/data/szr207/projects/ArqMath/runs/es.rr.1k.dat'

In [101]:
ans_path = '/data/szr207/dataset/ArqMath/jsons/answers/all.ans.jsonl'
ques_path = '/data/szr207/dataset/ArqMath/jsons/questions/all.ques.jsonl'

In [102]:
dict_ans = {}

with jsonlines.open(os.path.join(ans_path)) as reader:
        for obj in tqdm(reader):
            dict_ans[obj['post_id']] = obj

In [24]:
import xml.etree.ElementTree as ET
import re
class Topic:
    """
    This class shows a topic for task 1. Each topic has an topic_id which is str, a title and question which
    is the question body and a list of tags.
    """

    def __init__(self, topic_id, title, question, tags):
        self.topic_id = topic_id
        self.title = title
        self.question = question
        self.lst_tags = tags


class TopicReader:
    """
    This class takes in the topic file path and read all the topics into a map. The key in this map is the topic id
    and the values are Topic which has 4 attributes: id, title, question and list of tags for each topic.

    To see each topic, use the get_topic method, which takes the topic id and return the topic in Topic object and
    you have access to the 4 attributes mentioned above.
    """

    def __init__(self, topic_file_path):
        self.__map_topics = self.__read_topics(topic_file_path)

    def __read_topics(self, topic_file_path):
        map_topics = {}
        tree = ET.parse(topic_file_path)
        root = tree.getroot()
        for child in root:
            topic_id = child.attrib['number']
            title = child[0].text
            question = child[1].text
            lst_tag = child[2].text.split(",")
            map_topics[topic_id] = Topic(topic_id, title, question, lst_tag)
        return map_topics

    def get_topic(self, topic_id):
        if topic_id in self.__map_topics:
            return self.__map_topics[topic_id]
        return None

def remove_stop(query):
    with open('englishST.txt') as f:
        all_stopwords = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    all_stopwords = [x.strip() for x in all_stopwords] 
    text_tokens = query.split(' ')
    query = [word for word in text_tokens if not word in all_stopwords]
    query = ' '.join(query)
    return query


def remove_punct(my_str):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    # To take input from the user
    # my_str = input("Enter a string: ")

    # remove punctuation from the string
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct


#"In this example, the title and the question body of topic with id A.1 is printed."
topic_file_path = "/data/szr207/dataset/ArqMath/Task1/Sample Topics/Task1_Samples_V2.0.xml"
topic_reader = TopicReader(topic_file_path)
# with open('es.punct.dat', 'w') as eval_file:
query_topic = []
for topic_id in ['A.31','A.78', 'A.101']:
    #     topic_id = "A.31"
    title = re.sub('<[^<]+?>', '', topic_reader.get_topic(topic_id).title)
    body = topic_reader.get_topic(topic_id).question
    body_pro = re.sub('<[^<]+?>', '', body)
    query = title + '. ' + body_pro
    query_topic.append(reduce_str(query,100))

In [103]:
from collections import defaultdict
f = open(path_run, "r")
dict_qid_aid = defaultdict(list)
for i in f.readlines():
    row = i.rstrip().split('\t')
    dict_qid_aid[row[0]].append(row[2])

In [104]:
# with open('es.rr.roberta.dat', 'w') as eval_file:
dict_q_sortans = defaultdict(list)

for topic_id, ques,qid in zip(['A.31','A.78', 'A.101'], query_topic, dict_qid_aid):
    count = 1
    print(topic_id)
    for ans_id in dict_qid_aid[qid]:
        try:
            ans = reduce_str(re.sub('<[^<]+?>', '',dict_ans[int(ans_id)]['body']), 200)
            tokens = roberta.encode(ques, ans)
            prediction = roberta.predict('sentence_classification_head', tokens).cpu().detach().numpy()[0][1]
        except:
            try :
                print('more than 100 token')
                ans = reduce_str(re.sub('<[^<]+?>', '',dict_ans[int(ans_id)]['body']), 100)
                tokens = roberta.encode(ques, ans)
                prediction = roberta.predict('sentence_classification_head', tokens).cpu().detach().numpy()[0][1]
            except :
                try :
                    print('more than 70 token')
                    ans = reduce_str(re.sub('<[^<]+?>', '',dict_ans[int(ans_id)]['body']), 70)
                    tokens = roberta.encode(ques, ans)
                    prediction = roberta.predict('sentence_classification_head', tokens).cpu().detach().numpy()[0][1]
                except :
                    print('more than 200')
                    ans = re.sub('<[^<]+?>', '',dict_ans[int(ans_id)]['body'])[:200]
                    tokens = roberta.encode(ques, ans)
                    prediction = roberta.predict('sentence_classification_head', tokens).cpu().detach().numpy()[0][1]
#         print('A.31', 1, ans_id, count, prediction, 'rr')
        dict_q_sortans[topic_id].append((ans_id, prediction))
#         eval_file.write(topic_id+'\t'+ '1\t' +ans_id+'\t'+str(count)+'\t'+ str(prediction)+'\t'+ 'run_1'+'\n')
        
        count+=1

A.31
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 70 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 100 token
more than 70 token
more than 100 tok

In [105]:
dict_q_sortans

defaultdict(list,
            {'A.31': [('2246314', -3.410285),
              ('134854', -3.990594),
              ('2170920', -3.18266),
              ('2451383', -3.3835),
              ('1443248', -2.6832108),
              ('1797473', -3.2536602),
              ('2593223', -2.6244898),
              ('832924', -2.1937146),
              ('2839560', -3.4844913),
              ('3022023', -3.6900697),
              ('2599825', -3.115055),
              ('2260940', -1.7033265),
              ('1886470', -3.7125232),
              ('1001158', -1.8847647),
              ('92249', -0.7929986),
              ('2236010', -1.4621124),
              ('50028', -3.5816994),
              ('2564691', -1.002713),
              ('1944794', -3.3106492),
              ('2676887', -3.383018),
              ('1077818', -1.5529995),
              ('1034536', -4.0982103),
              ('2385043', -3.232091),
              ('521180', -2.529091),
              ('2345997', -4.039093),
              ('175

In [106]:
def Sort_Tuple(tup):  
      
    # getting length of list of tuples 
    lst = len(tup)  
    for i in range(0, lst):  
          
        for j in range(0, lst-i-1):  
            if (tup[j][1] > tup[j + 1][1]):  
                temp = tup[j]  
                tup[j]= tup[j + 1]  
                tup[j + 1]= temp  
    return tup 

In [107]:
Sort_Tuple(dict_q_sortans['A.31'])[::-1]

[('2236231', -0.10231972),
 ('669362', -0.11424506),
 ('1583217', -0.11457932),
 ('1886353', -0.13902092),
 ('2260955', -0.14249718),
 ('2915793', -0.17104077),
 ('1273532', -0.19411564),
 ('1727954', -0.2005459),
 ('2417791', -0.23279488),
 ('2417892', -0.25730067),
 ('1811741', -0.26508898),
 ('2621225', -0.30834275),
 ('2843955', -0.32918322),
 ('281806', -0.35849327),
 ('1686054', -0.372527),
 ('2329617', -0.39655012),
 ('1583213', -0.39871722),
 ('1339353', -0.40179765),
 ('1923593', -0.41608196),
 ('2457727', -0.41952908),
 ('1927460', -0.4266832),
 ('2505574', -0.43935382),
 ('2472954', -0.5016792),
 ('541603', -0.5536638),
 ('256477', -0.59247386),
 ('2727406', -0.619092),
 ('1927474', -0.63039815),
 ('516957', -0.6338116),
 ('945492', -0.6422136),
 ('484770', -0.65048236),
 ('1429039', -0.65264654),
 ('784705', -0.6543317),
 ('898625', -0.65519536),
 ('629016', -0.6690767),
 ('547690', -0.67930675),
 ('1912332', -0.6860341),
 ('986645', -0.690143),
 ('1295122', -0.6925542),
 (

In [108]:
for qid in dict_q_sortans:
    dict_q_sortans[qid] = Sort_Tuple(dict_q_sortans[qid])[::-1]

In [109]:
dict_q_sortans

defaultdict(list,
            {'A.31': [('2236231', -0.10231972),
              ('669362', -0.11424506),
              ('1583217', -0.11457932),
              ('1886353', -0.13902092),
              ('2260955', -0.14249718),
              ('2915793', -0.17104077),
              ('1273532', -0.19411564),
              ('1727954', -0.2005459),
              ('2417791', -0.23279488),
              ('2417892', -0.25730067),
              ('1811741', -0.26508898),
              ('2621225', -0.30834275),
              ('2843955', -0.32918322),
              ('281806', -0.35849327),
              ('1686054', -0.372527),
              ('2329617', -0.39655012),
              ('1583213', -0.39871722),
              ('1339353', -0.40179765),
              ('1923593', -0.41608196),
              ('2457727', -0.41952908),
              ('1927460', -0.4266832),
              ('2505574', -0.43935382),
              ('2472954', -0.5016792),
              ('541603', -0.5536638),
              ('256477'

In [110]:
with open('es.rr.roberta.cp1.dat', 'w') as eval_file:
    for qid in dict_q_sortans:
        count = 1
        for ans,score in dict_q_sortans[qid]:
            eval_file.write(qid+'\t'+ '1\t' +ans+'\t'+str(count)+'\t'+ str(5+score)+'\t'+ 'run_1'+'\n')
            count += 1

In [3]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar -l2 -m P ../runs/qrel_task1 tf.psu-task1-rrf.prim.mlt.anserini.bert-auto-both-A.tsv

P_5                   	all	0.1299
P_10                  	all	0.1429
P_15                  	all	0.1325
P_20                  	all	0.1227
P_30                  	all	0.1130
P_100                 	all	0.0751
P_200                 	all	0.0546
P_500                 	all	0.0234
P_1000                	all	0.0117


In [4]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar -l2 -m ndcg ../runs/qrel_task1 tf.psu-task1-prim.mlt-auto-both-A.tsv

ndcg                  	all	0.2712


In [5]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar -l2  ../runs/qrel_task1 tf.psu-task1-prim.mlt-auto-both-A.tsv

runid                 	all	mlt_base
num_q                 	all	77
num_ret               	all	10757
num_rel               	all	1804
num_rel_ret           	all	781
map                   	all	0.0857
gm_map                	all	0.0117
Rprec                 	all	0.1111
bpref                 	all	0.0854
recip_rank            	all	0.2511
iprec_at_recall_0.00  	all	0.2982
iprec_at_recall_0.10  	all	0.2043
iprec_at_recall_0.20  	all	0.1460
iprec_at_recall_0.30  	all	0.1197
iprec_at_recall_0.40  	all	0.0967
iprec_at_recall_0.50  	all	0.0815
iprec_at_recall_0.60  	all	0.0693
iprec_at_recall_0.70  	all	0.0449
iprec_at_recall_0.80  	all	0.0257
iprec_at_recall_0.90  	all	0.0087
iprec_at_recall_1.00  	all	0.0067
P_5                   	all	0.1351
P_10                  	all	0.1325
P_15                  	all	0.1342
P_20                  	all	0.1253
P_30                  	all	0.1130
P_100                 	all	0.0769
P_200                 	all	0.0501
P_500                 	all	0.0203
P_1000                

In [6]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar -l2 ../runs/qrel_task1  tf.psu-task1-prim.anserini-auto-both-A.tsv

runid                 	all	anserini
num_q                 	all	77
num_ret               	all	10502
num_rel               	all	1804
num_rel_ret           	all	660
map                   	all	0.0784
gm_map                	all	0.0087
Rprec                 	all	0.1118
bpref                 	all	0.0926
recip_rank            	all	0.2858
iprec_at_recall_0.00  	all	0.3123
iprec_at_recall_0.10  	all	0.1972
iprec_at_recall_0.20  	all	0.1389
iprec_at_recall_0.30  	all	0.1105
iprec_at_recall_0.40  	all	0.0844
iprec_at_recall_0.50  	all	0.0638
iprec_at_recall_0.60  	all	0.0449
iprec_at_recall_0.70  	all	0.0245
iprec_at_recall_0.80  	all	0.0087
iprec_at_recall_0.90  	all	0.0072
iprec_at_recall_1.00  	all	0.0058
P_5                   	all	0.1610
P_10                  	all	0.1390
P_15                  	all	0.1247
P_20                  	all	0.1143
P_30                  	all	0.1082
P_100                 	all	0.0695
P_200                 	all	0.0429
P_500                 	all	0.0171
P_1000                